In [2]:
from datetime import datetime
from os.path import join
from os import listdir
import re #for camel case conversion
import json
import ast

import numpy as np
import pandas as pd
from pyarrow.parquet import ParquetFile
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
# from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
#     convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

In [3]:
def camel_case(s):
  s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].lower(), s[1:]])

def sherlock_case(s):
    s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
    s = ''.join([s[0].lower(), s[1:]])
    s = ''.join(map(lambda x: x if x.islower() else " "+x, s))
    return s

In [6]:
path_raw = '../data/data/raw'
path_processed = '../data/data/processed'
path_out_true_types = '../../combined/results/true_types'
path_out_predictions = '../../combined/results/predictions'

In [7]:
test_values = pd.read_parquet(join(path_raw, "test_values.parquet"))
train_values = pd.read_parquet(join(path_raw, "train_values.parquet"))
validation_values = pd.read_parquet(join(path_raw, "val_values.parquet"))

# test_labels = pd.read_parquet(join(path_raw, "test_labels.parquet"))
# train_labels = pd.read_parquet(join(path_raw, "train_labels.parquet"))
validation_labels = pd.read_parquet(join(path_raw, "val_labels.parquet"))

# test_processed = pd.read_parquet(join(path_processed, "test.parquet"))
# train_processed = pd.read_parquet(join(path_processed, "train.parquet"))
validation_processed = pd.read_parquet(join(path_processed, "validation.parquet"))

In [8]:
# values = pd.concat([test_values, train_values, validation_values], ignore_index = True, axis = 0)
# labels = pd.concat([test_labels, train_labels, validation_labels], ignore_index = True, axis = 0)
# processed = pd.concat([test_processed, train_processed, validation_processed], ignore_index = True, axis = 0)
labels = validation_labels
processed = validation_processed

In [10]:
# values
# print(type(values)) 
# print(len(values))
print(len(processed))
# labels.head(10)
print(len(labels))
# Write out true types for dataset
labels.to_parquet(join(path_out_true_types, "sherlock_validation.parquet"))

137353
137353


### Sherlock

In [11]:
model = SherlockModel();
# model.model_files_directory = '../../../sherlock/model_files'
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

W0516 22:57:23.683773 140402237970240 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0516 22:57:23.684644 140402237970240 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0516 22:57:23.686903 140402237970240 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Varia

In [12]:
predicted_labels = model.predict(processed, "sherlock")
predicted_labels = pd.DataFrame(predicted_labels)

In [13]:
predicted_labels.columns = ['type']
predicted_labels.to_parquet(join(path_out_predictions, "sherlock_sherlock_validation.parquet"))

### Sato, NOT USABLE SEE SATO FOLDER FOR EXTRACTION

In [10]:
# TYPENAME = os.environ['TYPENAME']
# valid_types = get_valid_types(TYPENAME)
# print(valid_types)
# label_enc = LabelEncoder()
# label_enc.fit(valid_types)

# MAX_COL_COUNT = 10
# topic_dim = 400
# pre_trained_loc = './pretrained_sato'
# # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # print("PyTorch device={}".format(device))
# device = torch.device('cpu')

['address', 'affiliate', 'affiliation', 'age', 'album', 'area', 'artist', 'birthDate', 'birthPlace', 'brand', 'capacity', 'category', 'city', 'class', 'classification', 'club', 'code', 'collection', 'command', 'company', 'component', 'continent', 'country', 'county', 'creator', 'credit', 'currency', 'day', 'depth', 'description', 'director', 'duration', 'education', 'elevation', 'family', 'fileSize', 'format', 'gender', 'genre', 'grades', 'isbn', 'industry', 'jockey', 'language', 'location', 'manufacturer', 'name', 'nationality', 'notes', 'operator', 'order', 'organisation', 'origin', 'owner', 'person', 'plays', 'position', 'product', 'publisher', 'range', 'rank', 'ranking', 'region', 'religion', 'requirement', 'result', 'sales', 'service', 'sex', 'species', 'state', 'status', 'symbol', 'team', 'teamName', 'type', 'weight', 'year']


In [11]:
# feature_group_cols = {}
# sherlock_feature_groups = ['char', 'word', 'par', 'rest']
# for f_g in sherlock_feature_groups:
#     feature_group_cols[f_g] = list(pd.read_csv(join(os.environ['BASEPATH'],
#                                           'configs', 'feature_groups', 
#                                           "{}_col.tsv".format(f_g)),
#                                            sep='\t', header=None, 
#                                            index_col=0)[1])

In [12]:
# pad_vec = lambda x: np.pad(x, (0, topic_dim - len(x)),
#                     'constant',
#                     constant_values=(0.0, 1/topic_dim))

In [13]:
# classifier = models_sherlock.build_sherlock(sherlock_feature_groups, num_classes=len(valid_types), topic_dim=topic_dim, dropout_ratio=0.35)
# #classifier.load_state_dict(torch.load(join(pre_trained_loc, 'sherlock_None.pt'), map_location=device))
# model = CRF(len(valid_types) , batch_first=True).to(device)
# #model.load_state_dict(torch.load(join(pre_trained_loc, 'model.pt'), map_location=device))

# loaded_params = torch.load(join(pre_trained_loc, 'model.pt'), map_location=device)
# classifier.load_state_dict(loaded_params['col_classifier'])
# model.load_state_dict(loaded_params['CRF_model'])

# classifier.eval()
# model.eval()

FileNotFoundError: [Errno 2] No such file or directory: './pretrained_sato/model.pt'

In [ ]:
# def extract(df):

#     df_dic = {'df':df, 'locator':'None', 'dataset_id':'None'}
#     feature_dic = {}
#     n = df.shape[1]

#     # topic vectors
#     topic_features = extract_topic_features(df_dic)
#     topic_vec = pad_vec(topic_features.loc[0,'table_topic'])
#     feature_dic['topic'] = torch.FloatTensor(np.vstack((np.tile(topic_vec,(n,1)), np.zeros((MAX_COL_COUNT - n, topic_dim)))))


#     # sherlock vectors
#     sherlock_features = extract_sherlock_features(df_dic)
#     print(sherlock_features.head())
#     for f_g in feature_group_cols:
#         temp = sherlock_features[feature_group_cols[f_g]].to_numpy()
#         temp = np.vstack((temp, np.zeros((MAX_COL_COUNT - n, temp.shape[1])))).astype('float')
#         temp = np.nan_to_num(temp)
#         feature_dic[f_g] = torch.FloatTensor(temp)

#     # dictionary of features, labels, masks
#     return feature_dic, np.zeros(MAX_COL_COUNT), torch.tensor([1]*n + [0]*(MAX_COL_COUNT-n), dtype=torch.uint8)

In [ ]:
# def evaluate(df):

#     feature_dic, labels, mask = extract(df)

#     emissions = classifier(feature_dic).view(1, MAX_COL_COUNT, -1)
#     mask = mask.view(1, MAX_COL_COUNT)
#     pred = model.decode(emissions, mask)[0]

#     return label_enc.inverse_transform(pred)

In [ ]:
# predicted_labels = pd.DataFrame(columns=['type'])

In [ ]:
# for idx, col in enumerate(values['values']):
#     col = ast.literal_eval(col)
#     df = pd.DataFrame(columns=['col'], data=col)
#     prediction = evaluate(df)
#     predicted_labels['type'].append(prediction)
#     if(idx % 1000 == 0):
#         break;